In [151]:
import numpy as np
from skimage.measure import label, regionprops
from skimage.io import imread
from skimage.filters import threshold_yen
import napari
import glob

In [152]:
def read_sequence(exp, time=0, slice=0, volume=True):
    if volume:
        path = '../MasterThesisData/' + exp + '/entry' + str(time).zfill(4) + '_no_extpag_db0100_vol/'
    else:
        path = '../MasterThesisData/slice ' + str(slice) + '/'
        
    image = imread(path+'entry' + str(time).zfill(4) + '_no_extpag_db0100_vol_' + str(slice).zfill(6) + '.tiff')
    files_number = len(glob.glob1(path,"*.tiff"))
    sequence = np.zeros((files_number,image.shape[0],image.shape[1]))

    for i in range(files_number):
        if volume:
            image = imread(path+'entry'+str(time).zfill(4)+'_no_extpag_db0100_vol_'+str(i).zfill(6)+'.tiff')
        else:
            image = imread(path+'entry'+str(i).zfill(4)+'_no_extpag_db0100_vol_'+str(slice).zfill(6)+'.tiff')
        sequence[i,:,:] = image
    return sequence



def isAgglomerate(rps, i, smallest_area, eccentricity, volume=False):
    # Check if the region is too small
    if rps[i].area < smallest_area:
        return False
    # Check if the region is rounded (axis_major_length vs axis_minor_length) (area vs area_convex) (axis_major_length vs equivalent_diameter_area) (eccentricity vs threshold)
    if not volume:
        if rps[i].eccentricity > eccentricity:
            return False
    return True



def segment(image, n_agglomerates=50, smallest_area=5, eccentricity=0.99):
    
    image = (image - np.min(image))/(np.max(image) - np.min(image))

    threshold = threshold_yen(image)
    mask = image > threshold
    mask_labeled = np.vectorize(label, signature='(n,m)->(n,m)')(mask)

    rps = regionprops(mask_labeled)
    areas = [r.area for r in rps]
    idxs = np.argsort(areas)[::-1]
    new_mask = np.zeros_like(mask_labeled)
    
    for j, i in enumerate(idxs[:n_agglomerates]):
        if isAgglomerate(rps, i, smallest_area, eccentricity):
            new_mask[tuple(rps[i].coords.T)] = j + 1
    return new_mask



def propagate_labels(mask, start=12, stop=0):
    for slice in range(start, mask.shape[0]-stop):
        previous_slice = mask[slice-1,:,:]
        current_slice = mask[slice,:,:]
        current_slice[current_slice > 0] = current_slice[current_slice > 0] + np.max(previous_slice)
        flag = False

        for previous_slice_label in np.unique(previous_slice):
            if previous_slice_label == 0:
                continue
            previous_slice_region = previous_slice == previous_slice_label
            overlap = current_slice * previous_slice_region
            unique_labels = np.unique(overlap)
            
            for _, current_slice_label in enumerate(unique_labels):
                if current_slice_label == 0:
                    continue
                temp = np.array([[previous_slice_label, current_slice_label, len(overlap[overlap == current_slice_label])]]).T
                if not flag:
                    mapping = temp
                    flag = True
                else:
                    mapping = np.append(mapping, temp, axis=1)
        
        for current_slice_label in np.unique(mapping[1,:]):
            temp = mapping[:,mapping[1,:] == current_slice_label]
            previous_slice_label = temp[0, np.argmax(temp[2,:])]
            current_slice[current_slice == current_slice_label] = previous_slice_label
        
        mask[slice,:,:] = current_slice
            
    return mask

In [153]:
sequence = read_sequence('P28A_FT_H_Exp1', time=150)
viewer = napari.Viewer()
viewer.add_image(sequence)

<Image layer 'sequence' at 0x2ec211040>

In [154]:
segmented_image = (np.zeros_like(sequence)).astype(int)
for i in range(sequence.shape[0]):
    segmented_image[i,:,:] = segment(sequence[i,:,:])

new_segmented_image = propagate_labels(segmented_image)
viewer.add_labels(new_segmented_image)

<Labels layer 'new_segmented_image' at 0x2ec347f70>

In [155]:
background, labels = viewer.layers
background.opacity = 0.4
labels.blending = 'additive'
labels.opacity = 0.8